In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import gzip
import json
from transformers import pipeline
import math

In [50]:
raw = pd.read_json('modcloth_final_data.json.gz',lines=True)

In [51]:
raw

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,Emily,just right,small,991571,NaN,NaN,NaN,NaN
1,123373,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,sydneybraden2001,just right,small,587883,NaN,NaN,NaN,NaN
2,123373,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,Ugggh,slightly long,small,395665,9.0,NaN,NaN,NaN
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,alexmeyer626,just right,fit,875643,NaN,NaN,NaN,NaN
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,dberrones1,slightly long,small,944840,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,807722,NaN,8,4.0,b,NaN,36.0,outerwear,NaN,5ft 8in,Jennifer,just right,fit,727820,8.5,average,Cute jacket!,Cute jacket!
82786,807722,NaN,12,5.0,ddd/f,NaN,34.0,outerwear,NaN,5ft 5in,Kelli,slightly long,small,197040,NaN,NaN,It's a beautiful jacket.,It's a beautiful jacket. I love how it's knit ...
82787,807722,NaN,12,5.0,dddd/g,36.0,32.0,outerwear,NaN,5ft 4in,elacount,just right,fit,102493,NaN,NaN,I love this blazer. It is,I love this blazer. It is a great office piece...
82788,807722,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,5ft 3in,jennaklinner,just right,fit,756491,NaN,NaN,I love this blazer!! I wo,I love this blazer!! I wore it yesterday and g...


In [52]:
sentiment_analysis = pipeline("sentiment-analysis")
def review_text_transform(x):
    if not isinstance(x, str): return 0.5
    if len(x) >= 512 : return 0.5
    label = sentiment_analysis(x)[0]['label']
    if label == 'POSITIVE':
        return 1
    else:
        return 0

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [53]:
for i in range(len(raw['review_text'])):
    raw['review_text'][i] = review_text_transform(raw['review_text'][i])

C:\Users\stanl\AppData\Local\Temp\ipykernel_17192\401756763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['review_text'][i] = review_text_transform(raw['review_text'][i])


In [56]:
raw.to_csv('sentiment.csv')

In [81]:
raw = pd.read_csv('sentiment.csv',index_col = False)

c:\software\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [82]:
raw = raw.drop(['user_name','length','review_summary','Unnamed: 0'],axis=1)

In [83]:
raw

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,fit,user_id,shoe size,shoe width,review_text
0,123373,29.0,7,5.0,d,38.0,34.0,new,36.0,5ft 6in,small,991571,NaN,NaN,0.5
1,123373,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,small,587883,NaN,NaN,0.5
2,123373,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,small,395665,9.0,NaN,0.5
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,fit,875643,NaN,NaN,0.5
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,small,944840,NaN,NaN,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,807722,NaN,8,4.0,b,NaN,36.0,outerwear,NaN,5ft 8in,fit,727820,8.5,average,1.0
82786,807722,NaN,12,5.0,ddd/f,NaN,34.0,outerwear,NaN,5ft 5in,small,197040,NaN,NaN,1.0
82787,807722,NaN,12,5.0,dddd/g,36.0,32.0,outerwear,NaN,5ft 4in,fit,102493,NaN,NaN,1.0
82788,807722,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,5ft 3in,fit,756491,NaN,NaN,1.0


In [60]:
raw['waist'].isnull().sum()

79908

In [61]:
raw['size'].isnull().sum()

0

In [62]:
raw['quality'].isnull().sum()

68

In [63]:
raw['cup size'].isnull().sum()

6255

In [64]:
raw['hips'].isnull().sum()

26726

In [65]:
raw['bra size'].isnull().sum()

6018

In [66]:
raw['category'].isnull().sum()

0

In [67]:
raw['bust'].isnull().sum()

70936

In [68]:
raw['height'].isnull().sum()

1107

In [69]:
raw['shoe size'].isnull().sum()

54875

In [70]:
raw['shoe width'].isnull().sum()

64183

In [92]:
sum(raw['fit']=='fit')

56757

In [105]:
bust_size_one_hot = pd.get_dummies(raw['bust size'])

In [106]:
rented_for_one_hot = pd.get_dummies(raw['rented for'])

In [107]:
body_type_one_hot = pd.get_dummies(raw['body type'])

In [108]:
category_one_hot = pd.get_dummies(raw['category'])